# Create an executable PDF

Let's create a PDF file that you can view in a PDF viewer and that you can also run as an executable file in its own right.

We're going to use 'actually portable executable', so download libcosmopolitan and we'll use that to compile a 'hello world' program for the pdf file.

In [ ]:
import os
!mkdir -p cosmocc
os.chdir('cosmocc')
!wget https://cosmo.zip/pub/cosmocc/cosmocc.zip
!unzip cosmocc.zip
os.chdir('..')

Create a little program to embed in the pdf

In [38]:
c = """// hello.c
#include <stdio.h>

int main() {
  printf("I'm also a little Computer Program!\\n");
}
"""
outfile = open("hello.c",'w')
outfile.write(c)
outfile.flush()

In [39]:
!./cosmocc/bin/cosmocc -o hello hello.c

We're going to embed the executable file in an pdf object stream. The first 0x40 bytes will be the first 0x40 bytes of the 'hello' executable, then the OBJSTREAM_HEAD below, then the rest of the 'hello' executable, then OBJSTREAM_TAIL, then the rest of our valid PDF.

In [40]:
OBJSTREAM_HEAD = """%PDF-1.5
1 0 obj <<
/Length {}
>>
stream
"""

OBJSTREAM_TAIL = """endstream
endobj
"""

# Body of a minimal valid pdf.
min_pdf = """1 0 obj<</Type/Catalog/Pages 2 0 R>>endobj 2 0 obj<</Type/Pages/Kids[3 0 R]/Count 1>>endobj 
3 0 obj<</Type/Page/MediaBox[0 0 3 3]>>Hello there I'm a PDF. endobj
xref
0 4
0000000000 65535 f
0000000010 00000 n
0000000053 00000 n
0000000102 00000 n
trailer<</Size 4/Root 1 0 R>>
startxref
149
%EOF"""

min_text = """1 0 obj
  << /Type /Catalog
     /Pages 2 0 R
  >>
endobj

2 0 obj
  << /Type /Pages
     /Kids [3 0 R]
     /Count 1
     /MediaBox [0 0 300 144]
  >>
endobj

3 0 obj
  <<  /Type /Page
      /Parent 2 0 R
      /Resources
       << /Font
           << /F1
               << /Type /Font
                  /Subtype /Type1
                  /BaseFont /Times-Roman
               >>
           >>
       >>
      /Contents 4 0 R
  >>
endobj

4 0 obj
  << /Length 55 >>
stream
  BT
    /F1 18 Tf
    0 0 Td
    (I'm a little PDF file!) Tj
  ET
endstream
endobj

xref
0 5
0000000000 65535 f 
0000000018 00000 n 
0000000077 00000 n 
0000000178 00000 n 
0000000457 00000 n 
trailer
  <<  /Root 1 0 R
      /Size 5
  >>
startxref
565
%%EOF"""

Construct our PDF/executable file.

In [41]:
import os

hfile = open("hello", 'rb')
outfile = open("hello.pdf", 'wb')

hfile.seek(0, 0)
outfile.write(hfile.read(0x40))

# Get the length of the length string
stub_size = os.stat("hello").st_size - 0x40
header_len = len(OBJSTREAM_HEAD) + (len(str(stub_size)) - 2)
stream_len = stub_size - header_len
outfile.write(OBJSTREAM_HEAD.format(stream_len).encode('utf−8'))

hfile.seek(0x40+header_len, 0)
outfile.write(hfile.read())
outfile.write(OBJSTREAM_TAIL.encode('utf−8'))

outfile.write(min_text.encode('utf-8'))

outfile.flush()
outfile.close()


View the pdf in a pdf viewer

In [42]:
!evince hello.pdf

Run the pdf as an executable

In [43]:
!chmod +x hello.pdf
!./hello.pdf

I'm also a little Computer Program!
